<a href="https://colab.research.google.com/github/hainesdata/gas/blob/main/lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Imports
import pandas as pd
import plotly.express as px
import numpy as np
import datetime
import requests
import regex as re
import random
import time
import sys
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statistics import stdev, mean, median, variance
from math import sqrt
from bs4 import BeautifulSoup
from requests.exceptions import ProxyError

In [ ]:
# Load raw gasbuddy dataset
raw = pd.read_csv('gas_buddy_2022-04-18.csv')

# Display variable properties
raw.info()
raw.nunique()

In [ ]:
# Create working copy of raw data to preserve raw data
sandbox = raw.copy()

# Instantiate LabelEncoder and StandardScaler objects from scikit-learn
le = LabelEncoder()
s = StandardScaler()

# Encode services feature, show distribution histogram (non ordinal)
sandbox['services_included'] = le.fit_transform(sandbox['services_included'])
px.histogram(x=sandbox['services_included'], height=300, width=500).show()

# Filter prices listed as 0, show boxplot histogram of payment type
sandbox = sandbox[sandbox['price_current'] != 0]
px.box(sandbox, x='payment_type', y='price_current', width=400, height=500).show()

# Show overall rating distribution
px.histogram(x=sandbox['overall_rating'], height=300, width=500).show()

# Show review count distribution
px.histogram(x=sandbox['review_count'], height=300, width=500).show()

# Encode station ID and show distribution
le.fit(sandbox[['loc_number']])
sandbox['loc_number'] = le.transform(sandbox[['loc_number']])
px.histogram(x=sandbox['loc_number'], height=300, width=500).show()

# Encode station name and show distribution
le.fit(sandbox[['loc_name']])
sandbox['loc_name'] = le.transform(sandbox[['loc_name']])
px.histogram(x=sandbox['loc_name'], height=300, width=500).show()

# Print number of unique cities
print(len(sandbox['city'].unique()))

# Encode city and show distribution
le.fit(sandbox[['city']])
sandbox['city'] = le.transform(sandbox[['city']])
px.histogram(x=sandbox['city'], height=300, width=500).show()

# Product name distribution
px.box(sandbox, x='product_name', y='price_current', width=600).show()

# Current price distribution
px.histogram(sandbox, x='price_current', color='product_name', barmode='stack', nbins=64, width=1000).show()

# Price vs review count relationship
px.scatter(sandbox, x='review_count', y='price_current', trendline='ols', width=900).show()

# Price vs rating relationship
px.scatter(sandbox, x='overall_rating', y='price_current', trendline='ols', width=900).show()

# Review vs rating count relationship
px.scatter(sandbox, x='overall_rating', y='review_count', trendline='ols', width=900).show()

# Price vs coordinate relationships 
px.scatter(sandbox, x='latitude', y='price_current', trendline='ols', width=900).show()
px.scatter(sandbox, x='longitude', y='price_current', trendline='ols', width=900).show()

# Station location map
fig = px.scatter_mapbox(sandbox, lat="latitude", lon="longitude", 
                        hover_name='loc_name',
                        hover_data=["latitude","longitude"],
                        zoom=4, height=500, width=400
                        )
fig.update_layout(mapbox_style="open-street-map")
fig.show()

# Location of stations with price less than USD $5, colored by product name
fig = px.scatter_mapbox(sandbox[sandbox['price_current'] < 5], lat="latitude", lon="longitude", 
                        hover_name='loc_name',
                        hover_data=["latitude","longitude"],
                        zoom=4, height=500, width=400,
                        color='product_name'
                        )
fig.update_layout(mapbox_style="open-street-map")
fig.show()

# Location name vs price relationship
px.scatter(sandbox, x='loc_name', y='price_current', width=900)

In [ ]:
# Reset working dataset
sandbox = raw.copy()

# Drop features and exclude price zeros
sandbox.drop(columns=['source_url', 'DATE_SCRAPED', 'RUN_START_DATE', 
                      'zip_code_searched', 'country', 'currency', 'state',
                      'address_1', 'address_2', 'services_included', 
                      'price_time_stamp'
                      ],
             inplace=True)
sandbox = sandbox[sandbox['price_current'] != 0]

# Print number of unique values per feature and feature properties
print(sandbox.nunique())
print(sandbox.info())

# Label-encode all features in input dataset that are strings
def label_encode(df):
    le = LabelEncoder()
    for col in df.columns:
        if type(df[col][0]) is str:
            le = le.fit(df[[col]].values.ravel())
            df[col] = le.transform(df[[col]].values.ravel())

# (Unused, here for experimenting) One-hot encode string features given selected features passed to cols parameter
def one_hot(df, cols):
    ohe = OneHotEncoder()
    for col in cols:
        if type(df[col][0]) is str:
            ohe = ohe.fit(df[[col]])
            enc_arr = ohe.transform(df[[col]]).toarray()
            onehot_df = pd.DataFrame(enc_arr, columns=ohe.get_feature_names_out([col]))
            if len(onehot_df.columns) > 5000:
                raise MemoryError(f'There are > 5000 columns in this encoded feature ({col}). Concatenating on input dataframe is expensive and may crash. Please reduce the number of columns for this feature or reduce the number of possible values for this feature.')
            df = df.drop(columns=[col])
            df = pd.concat([df, onehot_df], axis=1)
    return df

# Specify name of target variable and declare feature names as all columns excluding target name
y_name = 'price_current'
x_name = [name for name in sandbox.columns if name != y_name]

# Encode dataset
label_encode(sandbox)

# Drop NAs and display metadata for encoded dataset
sandbox = sandbox.dropna(how='any', axis=0)
sandbox.info()

# Train model
def train_model(df):
    # Specify name of target variable and declare feature names as all columns excluding target name
    y_name = 'price_current'
    x_name = [name for name in df.columns if name != y_name]
    X = df[x_name]
    y = df[y_name]

    # Train-test split
    X_t, X_v, y_t, y_v = train_test_split(X, y, test_size=0.2, random_state=42)

    # Instantiate linear regression and train model
    lr = LinearRegression()
    lr.fit(X_t, y_t)

    # Print feature coefficients
    weights = lr.coef_
    print('WEIGHTS---------------------')
    for i, w in zip(x_name, weights):
        print(f'{i.ljust(20)} {w}')
    print('')

    # Validate model
    y_hat = lr.predict(X_v)
    err = y_v-y_hat
    sigma_y = stdev(y_v)
    sigma_e = stdev(err)
    mse = mean_squared_error(y_v, y_hat)
    lmbda = 1
    me = sqrt(mse)
    pe = me/(2*lmbda*sigma_y)
    performance = me/(2*lmbda*sigma_e)

    # Display model performance statistics
    print('PERFORMANCE----------------')
    print(f'{"Mean Square Error".ljust(20)} {mse}')
    print(f'{"Mean Error".ljust(20)} {me}')
    print(f'{"Mean Percent Error".ljust(20)} {pe}')
    print(f'{"Error Variance".ljust(20)} {variance(err)}')
    print(f'{"Adj MSE Performance".ljust(20)} {performance}')
    px.histogram(x=err, width=900).show()

    # Examine relationship between performance variable and standard deviation
    x_p = []
    y_p = []
    for i in range(1, 5):
        x_p.append(i)
        y_p.append(me/(2*i*sigma_y))
    x_e = []
    y_e = []
    for i in range(1, 5):
        x_e.append(i)
        y_e.append(me/(2*i*sigma_e))
    fig = px.line(x=x_p, y=[y_p, y_e], labels={'x':'Lambda', 'value': 'Value'}, width=900)
    newnames = {'wide_variable_0':'MPE', 'wide_variable_1': 'Adjusted MSE'}
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                          legendgroup = newnames[t.name],
                                          hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                        )
                      )
    fig.show()
    return lr

# Execute model training function
model = train_model(sandbox)

In [ ]:
# Get list of ten working proxies for webscraping
def get_proxies():
    # Get proxies from proxyscrape
    url = 'https://gasbuddy.com'
    proxies_url = 'https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=10000&country=all&ssl=all&anonymity=all'
    response = requests.get(proxies_url)
    proxies = response.text.split('\r\n')

    # Test proxies and return first one working
    for proxy in proxies:
        try:
            r = requests.get(url, proxies={'http': proxy, 'https': proxy}, timeout=5)
            check = u'\u2713'
            print(f'{check}\t Proxy {proxy} is WORKING')
            return proxy
        except:
            check = u'\u2717'
            print(f'{check}\t Proxy {proxy} is not working')
    raise ProxyError('Proxy list exhausted, no working proxies found.')
    
# Load a given page using proxy
def load_page(url, proxy='none'):
    # Define header
    hdr = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5'
    }    

    # If 'none' is passed into the proxy parameter, perform request without proxy. Otherwise, the proxy address passed is used
    if proxy == 'none':
        resp = requests.get(url, headers=hdr)
        return resp
    resp = requests.get(url, headers=hdr, proxies={'http':f'{proxy}', 'https':f'{proxy}'})
    return resp

# Get station IDs for zipcode passed into zipcode parameter
def get_ids(zipcode, proxy, debug=True):
    # Get Gasbuddy search results
    url = f"https://www.gasbuddy.com/home?search={zipcode}&fuel=1&method=all&maxAge=24"
    resp = load_page(url, proxy)
    soup = BeautifulSoup(resp.text, "html.parser")

    # Prints response for debug
    if debug:
        print(resp)
    
    # Get IDs from search results
    u = []
    ids = soup.select('div[class*="GenericStationListItem-module__station___"]')
    for i in ids:
        u.append(i.get('id'))
    return u

# Parse gas station info
def get_info(id, city, proxy):
    # Load gas station page
    url = f"https://www.gasbuddy.com/station/{id}"
    resp = load_page(url, proxy)
    soup = BeautifulSoup(resp.text, "html.parser")

    # Parse zipcode
    features = []
    zip_match = re.search(r'CA,([0-9]{5})', str(soup.select('a[class*="Station-module__directionsLink___"]')[0].get('href')))
    postal_code = zip_match.group(1) if zip_match else None
    features.append(postal_code)

    # Parse location name
    try:
        loc_name = [item.text.split("\xa0")[0] for item in soup.select('h2[class*="StationInfoBox-module__header___"]')][0]
    except IndexError:
        try:
            loc_name = [item.text for item in soup.select('h2[class*="StationInfoBox-module__header___"]')]
        except IndexError:
            print('Error parsing loc_name: no stations exist in the zipcode or zipcode does not exist. Skipping...')
            return 0
    features.append(loc_name)

    # Get city from city parameter and add to features
    features.append(city)

    # Parse number of reviews on station
    review_count = int([re.split("[()]+", item.text)[1] for item in soup.select('span[class*="StationInfoBox-module__ratings___"]')][0])
    features.append(review_count)

    # Parse station latitude
    try:
        latitude = float(str(soup.select('a[class*="Station-module__directionsLink___"]')[0].get('href')).split('@')[1].split(',')[0])
    except:
        print(f'Error parsing latitude. Skipping...')
        return 0
    features.append(latitude)

    # Add station ID to features
    features.append(id)

    # Parse station phone number
    try:
        phone = [item.text for item in soup.select('a[class*="PhoneLink-module__blue___"]')][0]
    except IndexError:
        phone = ''
        print(f'Error parsing phone: Either phone was empty or incorrect. Skipping...')
    features.append(phone)

    # Parse station longitude
    try:
        longitude = float(str(soup.select('a[class*="Station-module__directionsLink___"]')[0].get('href')).split('@')[1].split(',')[1])
    except:
        print(f'Error parsing longitude. Skipping...')
        return 0
    features.append(longitude)

    # Gasbuddy shows credit price by default, so add credit to features as payment type
    payment_method = 'Credit'
    features.append(payment_method)

    # Parse station rating
    try:
        overall_rating = float([item.text for item in soup.select('span[class*="Station-module__ratingAverage___"]')][0])
    except ValueError:
        return 0
    features.append(overall_rating)

    # Create dictionary for gas type and respective price
    price_val = [item.text for item in soup.select('span[class*="FuelTypePriceDisplay-module__price___"]')]
    price_key = [item.text for item in soup.select('span[class*="GasPriceCollection-module__fuelTypeDisplay"]')]
    prices = {k:v for k,v in zip(price_key, price_val)}
    features.append(prices)

    return features

# Create test dataframe
df_test = pd.DataFrame(columns=['postal_code', 'loc_name', 'city', 'review_count', 'latitude', 'loc_number', 'phone', 'longitude', 'payment_type', 'overall_rating', 'prices'])

# Load California zipcodes and cities
zipcodes = pd.read_csv('zipcodes.us.csv', usecols=['state_code', 'zipcode', 'place'])
zipcodes = zipcodes[zipcodes['state_code'] == 'CA'].drop(columns=['state_code'])

# Specify number of zipcodes to search in
n = 25

# Scrape
for k, i in enumerate(zipcodes['zipcode'].sample(n+1)):
    print(f'({k}/{n})')
    print(f'[Zip: {i}] Retrieving IDs...')
    ids = get_ids(i)
    print(f'[Zip: {i}] Done.')
    failed = False
    for j in ids:
        print(f'[Zip: {i}] Retrieving ID {j} features...')
        features = get_info(j, zipcodes.iloc[k]['place'])
        if features == 0:
            failed = True
            break
        df_test.loc[len(df_test)] = features
        print(f'[Zip: {i}] Done.')
    if failed:
        print(f'[Zip: {i}] Parse failed.')
        continue
    print(f'[Zip: {i}] Parse successful.')
    cooldown = 5
    for t in range(cooldown, 0, -1):
        print(f"\rCooldown: {t}s", end='')
        time.sleep(1)
    print(f"\rCooldown: Done.", end='')
    print('\n')

# Export scraped data to CSV
df_test.to_csv('gasbuddy_test.csv', index=False)
print('Test data retrieved successfully and saved to gasbuddy_test.csv')

In [ ]:
# Load local test dataframe. If it doesn't exist in environment, load last exported CSV.
try:
    df_test_2 = df_test.copy()
except:
    df_test_2 = pd.read_csv('gasbuddy_test.csv')

# Unpack product prices
product_prices = pd.DataFrame(df_test_2['prices'].tolist()).stack().reset_index(level=1).rename(columns={0:'price_current'})
product_prices['product_name'] = product_prices['level_1']
product_prices = product_prices.drop('level_1', axis=1)
df_test_2 = pd.merge(df_test_2, product_prices, left_on=df_test_2.index, how='left', right_on=product_prices.index)
df_test_2 = df_test_2.drop(columns=['prices', 'key_0'])
df_test_2 = df_test_2[df_test_2['price_current'] != '- - -']
df_test_2['price_current'] = [float(i.replace('$','')) for i in df_test_2['price_current']]
df_test_2 = df_test_2.reset_index(drop=True)
df_test_2[:5]

# Preprocess validation data
y_name = 'price_current'
x_name = [name for name in sandbox.columns if name != y_name]
label_encode(df_test_2)
df_test_2 = df_test_2.dropna(how='any', axis=0)
print(df_test_2.info())


In [ ]:
# Run predictions and output error distribution
test_results = pd.DataFrame()
test_results['y_hat_test'] = model.predict(df_test_2[x_name])
test_results['y_test'] = df_test_2['price_current']
test_results['diff'] = test_results['y_test'] - test_results['y_hat_test']
px.histogram(test_results, x='diff', nbins=100, width=900, marginal='violin')